# Machine Learning - Assignment 2

## Decision tree induction algorithm for classification tasks

The aim of the assignment is to:

* Implement a decision tree induction algorithm for classification tasks.
* Make sure it works for real valued features and nominal features (categorical features without rank, e.g., red - blue - green).
* Test the algorithm on 3 datasets.

Follow the instructions and implement what is missing to complete the assignment. Some functions have been started to help you a little bit with the inputs or outputs of the function.

**Note:** You might need to go back and forth during your implementation of the code. The structure is set up to make implementation easier, but how you return values from the different functions might vary, and you might find yourself going back and change something to make it easier later on.

## Assignment preparations

We help you out with importing the libraries.

**IMPORTANT NOTE:** You may not import any more libraries than the ones already imported!

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Decision tree model

The main objective is to implement the decision tree model. The implemented decision tree needs to be recursive model, that is, it should be implemented general enough to call itself in order to grow. "Growing" a tree refers to the same thing as "training" a model.

As said in the introduction, the structure is set up to help with implementation, but the nature of this model makes it a bit harder to implement function-by-function. You will most likely go back and forth between these first tasks.

### 1) Grow Tree

We will start with the main function of the decision tree, the "growing" function. 

This function should be called when creating the model, but also from inside itself. It is responible for creating all the nodes and leafs in the tree.

In [9]:
def grow_tree(data,depth=0,max_depth=5):
   labels=data[:,-1]
   if stop_single_class(labels) or stop_max_depth(depth,max_depth):
      vals, counts= np.unique(labels,return_counts=True)
      return vals[np.argmax(counts)]
   best_gain,best_feature,best_value=find_best_split(data)
   if best_feature is not None and best_gain>0:
      left,right=split_data(data,best_feature,best_value)
      return{
         "feature": best_feature,
         "value": best_value,
         "left": grow_tree(left, depth + 1, max_depth, min_samples),
         "right": grow_tree(right, depth + 1, max_depth, min_samples)
         
      }
   vals, counts = np.unique(labels, return_counts=True)
   return vals[np.argmax(counts)]


### 2) Growth stopping conditions (or stopping criterias)

The "grow_tree" function needs some way of stop growing, otherwise it will grow indefinitely. We will adress this issue here.

The trees stopping criterias needs to handle the following:

1) When a node has only datapoints of a single class.

2) Prevent the tree from growing to large, i.e., a max depth.

3) Prevent the tree nodes from becoming to small.

4) Prevent the tree from growing when the node is large (has a lot of datapoints) but it is very unbalanced. This is an extention to case 1.

Can you think of some other stopping criterias that is good to have? 

In [10]:

def stop_single_class(current_data):

    if len(np.unique(current_data))==1:
        return True
    return False

def stop_max_depth(current_depth, max_depth):

    if current_depth>max_depth:
        return True
    return False
def stop_nodes_too_small(current_data,min_samples):
    if np.shape(current_data)[0]<min_samples:
        return True
    return False
def stop_unbalance(current_data,max_number_datapoints):
    pass
    
    

### 3) Best feature for splitting nodes

When we are growing the tree, we need to decide how we are going to split a node into two new nodes. This is achived by looking at the features of the data in the node and calculate the best feature to split on.

Here you have a choice:

* Split using **Information Entropy**
* Split using **Gini Impurity**

Finish the function below using Information Entropy or Gini Impurity.

**Note:** Your code should be able to handle both real and categorical features!

In [11]:
def gini_impurity(data):

  if data.size==0:
      return 0.0
  label_classifier, counts_classifier=np.unique(data[:,-1],return_counts=True)
  total=np.sum(counts_classifier)
  probability=counts_classifier/total
  sum_prob=np.sum(probability**2)
  return 1-sum_prob
def find_best_split(data):
    best_gain=0
    best_feature=None
    best_split_value=None
    parent_gini=gini_impurity(data)
    n_features=data[1]-1
    for index in range(n_features):
     feature_column=data[:,index]
     if isinstance(feature_column[1],(int,float,np.number)):
        threshold=[np.mean(feature_column)]
     elif isinstance(feature_column[1],str):
         threshold=np.unique(feature_column)
     for val in threshold:
            left,right=split_data(data,index,val)
            if len(left)==0 or len(right)==0:
                continue
            p=len(left)/len(data)
            gain = parent_gini - (p * gini_impurity(left) + (1 - p) * gini_impurity(right))
            if gain>best_gain:
                best_gain=gain
                best_feature=index
                best_split_value=val
    return best_gain,best_feature,best_split_value

### 4) Split data

When growing the tree, we need to split the data multiple times, and what we decide to split varies a lot. It is similar to splitting data into train and test sets (remember from assignment 1), but we split the data based on the best feature for growing a good tree.

**IMPORTANT NOTE:** To calculate binary splits for real-valued features, the following rule must be applied: an instance with a feature value lower than the mean feature value follows the left edge from the split node while all other instances follow the right edge from the split node.

In [12]:
def split_data(data,feature_index,split_value):

   feature_column= np.array(data[:,feature_index])
   if isinstance(data[1,feature_index],(int,float,np.number)):
     mean=np.mean(feature_column)
     left_split_mask=(feature_column<mean)
     right_split_mask=(feature_column>=mean)
   if isinstance(data[1,feature_index],str):
      left_split_mask= (feature_column== split_value)
      right_split_mask=(feature_column!=split_value)
   left_split=data[left_split_mask]
   right_split=data[right_split_mask]

   return left_split,right_split

In [26]:
data = np.array([
    [10, 'A', 0],
    [12, 'A', 0],
    [14, 'B', 1],
    [16, 'B', 1],
], dtype=object)


In [27]:
left, right = split_data(data, feature_index=1, split_value='A')

print("LEFT (categorical):")
print(left)

print("RIGHT (categorical):")
print(right)


LEFT (categorical):
[[10 'A' 0]
 [12 'A' 0]]
RIGHT (categorical):
[[14 'B' 1]
 [16 'B' 1]]


In [28]:
left, right = split_data(data, feature_index=0, split_value=13)

print("LEFT (numeric):")
print(left)

print("RIGHT (numeric):")
print(right)


LEFT (numeric):
[[10 'A' 0]
 [12 'A' 0]]
RIGHT (numeric):
[[14 'B' 1]
 [16 'B' 1]]


### 5) Predict with tree model

Finally, when we have grown our tree, we would like to use it for prediction. When using the tree for prediction, we traverse the tree for each datapoint untill we land in a leaf node.

In [13]:
def predict_with_tree(tree,sample):
    if not isinstance(tree, dict):
        return tree
    feature_index=tree["feature"]
    value=tree["value"]
    sample_value=sample[feature_index]
    if isinstance(sample[feature_index], (int,float,np.number)):
        go_left= sample_value<value
    else:
        go_left=sample_value==value
    if go_left:
        return predict_with_tree(tree["left"], sample)
    else:
        return predict_with_tree(tree["right"],sample)


## Test decision tree model, compare with scikit learn, and plot dataset results

In the last part of the lab, you are going to test your tree code and compare it to scikit learn. The goal is not to be better than an established library, but to give you an indication about if you are on the right track.

You will need to plot the results from your model and the scikit learn model using matplotlib. We suggest a simple but informative bar-charts.

To make the comparison fair, you should train and test both your decision tree algorithm and the scikit learn at least 5 times, and shuffle the data each time before splitting the data into a train and test set.

The datasets are:

* Wine - (https://archive.ics.uci.edu/dataset/109/wine)
* Heart disease - (https://archive.ics.uci.edu/dataset/45/heart+disease)
* Car - (https://archive.ics.uci.edu/dataset/19/car+evaluation)

**IMPORTANT NOTE 1:** Take note of the feature types in the datasets, some features are numerical in value but are in fact categorical features. Be sure to handle these features correctly!

**IMPORTANT NOTE 2:** In this assignment it helps to add an additional header with information about the features and if they are nominal (n) or real (r) features.

In [14]:
feature_types_car = [
    'n',  # buying
    'n',  # maint
    'n',  # doors
    'n',  # persons
    'n',  # lug_boot
    'n',  # safety
    'label'  # class
]

feature_types_heart = [
    'r',  # age
    'n',  # sex
    'n',  # cp
    'r',  # trestbps
    'r',  # chol
    'n',  # fbs
    'n',  # restecg
    'r',  # thalach
    'n',  # exang
    'r',  # oldpeak
    'n',  # slope
    'n',  # ca
    'n',  # thal
    'label'  # target
]

feature_types_wine = [
    'r',  # alcohol
    'r',  # malic acid
    'r',  # ash
    'r',  # alcalinity of ash
    'r',  # magnesium
    'r',  # total phenols
    'r',  # flavanoids
    'r',  # nonflavanoid phenols
    'r',  # proanthocyanins
    'r',  # color intensity
    'r',  # hue
    'r',  # OD280/OD315
    'r',  # proline
    'label'  # class
]


In [15]:
import pandas as pd
df = pd.read_csv("heart.csv")
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,52,1,0,125,212,0,1,168,0,1.0,2,2,3,0
1,53,1,0,140,203,1,0,155,1,3.1,0,0,3,0
2,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
3,61,1,0,148,203,0,1,161,0,0.0,2,1,3,0
4,62,0,0,138,294,1,1,106,0,1.9,1,3,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1
1021,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
1022,47,1,0,110,275,0,0,118,1,1.0,1,1,2,0
1023,50,0,0,110,254,0,0,159,0,0.0,2,0,2,1


In [16]:
df = pd.read_csv("wine.csv")
df

,alcohol,malic acid,ash,alcalinity of ash,magnesium,total phenols,flavanoids,nonflavanoid phenols,proanthocyanins,color intensity,hue,OD280/OD315 of diluted wines,proline,class
0,r,r,r,r,r,r,r,r,r,r,r,r,r,r
1,14.23,1.71,2.43,15.6,127,2.8,3.06,.28,2.29,5.64,1.04,3.92,1065,1
2,13.2,1.78,2.14,11.2,100,2.65,2.76,.26,1.28,4.38,1.05,3.4,1050,1
3,13.16,2.36,2.67,18.6,101,2.8,3.24,.3,2.81,5.68,1.03,3.17,1185,1
4,14.37,1.95,2.5,16.8,113,3.85,3.49,.24,2.18,7.8,.86,3.45,1480,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,13.71,5.65,2.45,20.5,95,1.68,.61,.52,1.06,7.7,.64,1.74,740,3
175,13.4,3.91,2.48,23,102,1.8,.75,.43,1.41,7.3,.7,1.56,750,3
176,13.27,4.28,2.26,20,120,1.59,.69,.43,1.35,10.2,.59,1.56,835,3
177,13.17,2.59,2.37,20,120,1.65,.68,.53,1.46,9.3,.6,1.62,840,3


In [17]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

You may use the "**accuracy_score**" function from scikit learn (imported above) to compare the performance of your own and scikit learns models.

See below for an example use.

In [18]:
y_true = [1,1,1,1,1] # Pretend labels
y_pred = [1,1,2,2,1] # Pretend prediction

accuracy_score(y_true, y_pred)

0.6

### 6) Dataset 1: Wine

In [23]:
data_wine = pd.read_csv("wine.csv", skiprows=[1]).to_numpy()


# TODO: Set up the data and split it into train and test-sets.
X = data_wine[:, :-1].astype(float)   # all columns except class
y = data_wine[:, -1]  

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, shuffle=True
)
print(X.dtype)
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

# TODO: Train and test your implemented tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Train and test scikit learns tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Do the above at least 5 times
# NOTE: Use loops here!

# TODO: Plot the results with matplotlib (plt)
# NOTE: One plot with all results is enough

float64
(124, 13) (54, 13)
(124,) (54,)


### 7) Dataset 2: Heart Disease

In [20]:
data_heart = pd.read_csv("heart.csv").to_numpy()

# TODO: Set up the data and split it into train and test-sets.

# TODO: Train and test your implemented tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Train and test scikit learns tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Do the above at least 5 times
# NOTE: Use loops here!

# TODO: Plot the results with matplotlib (plt)
# NOTE: One plot with all results is enough

### 8) Dataset 3: Car

In [21]:
data_car = pd.read_csv("car.csv").to_numpy()

# TODO: Set up the data and split it into train and test-sets.

# TODO: Train and test your implemented tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Train and test scikit learns tree model.
# NOTE: Use the same train/test split for your tree model and the scikit learn model

# TODO: Do the above at least 5 times
# NOTE: Use loops here!

# TODO: Plot the results with matplotlib (plt)
# NOTE: One plot with all results is enough

### 9) Training with normalized data on the wine-dataset

So far, we have trained our decision trees with "raw" data, i.e., we haven't done much preprocessing on the data.

Here we will do minor preprocessing on the data with the help of the scikit-learn library: https://scikit-learn.org/stable/modules/preprocessing.html

In [22]:
from sklearn import preprocessing

# TODO: Use the wine dataset from above and scale its features (Not the labels!) between 0 and 1

# TODO: Run the code from the dataset and compare the preprocessed vs non-preprocessed data.
# NOTE: You can copy most of the workflow from the dataset code above to save you some time.
# NOTE: Use the same train/test split for your tree model for both the preprocessed vs non-preprocessed data.

# TODO: Do the above at least 5 times
# NOTE: Use loops here!

# TODO: Plot the results with matplotlib (plt)
# NOTE: One plot with all results is enough

# Questions for examination:

In addition to completing the assignment with all its tasks, you should also prepare to answer the following questions:

1) Why is growing the tree indefinitely such a bad idea? The performance would increase would it not?

2) Beside preventing the tree from growing to large, what is the purpose of 'stopping criterias'?

3) What is the difference between **Information Entropy** and **Gini Impurity**?

4) What are some pros about using decision trees?

5) Did preprocessing the data help with performance when using decision trees?

# Finished!

Was part of the setup incorrect? Did you spot any inconsistencies in the assignment? Could something improve?

If so, please write them and send via email and send it to:

* marcus.gullstrand@ju.se

Thank you!